<a href="https://colab.research.google.com/github/sirrauf/Tugas-Course-Python-for-Data-Career-Analysis-Automation-to-Work-Faster-Smartpath/blob/main/For_Student_Study_Case_Dataset_2_2025_by_Ananda_Rauf_Maududi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Import Package & Load Database

In [ ]:
# Database Connection
db = sqlite3.connect('/content/nyc_taxi.db')
cursor = db.cursor()

OperationalError: unable to open database file

In [ ]:
# table list from the database
query = """
SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
"""

cursor.execute(query)
print(cursor.fetchall())

## 1. Table List

In [ ]:
# table trips
query = """
select * from trips limit 10;
"""
pd.read_sql_query(query, db)

In [ ]:
# table company
query = """
select * from company limit 10;
"""
pd.read_sql_query(query, db)

In [ ]:
# table zone
query = """
select * from zone limit 10;
"""
pd.read_sql_query(query, db)

## 2. Use Case

### 2.1 Daily Trip Count
The Story: The operations manager starts day by checking the volume from our key pickup zones for March 1st.

In [ ]:
# --- Q1: How many trips did we complete on March 1st, 2025? ---
query_1 = """
SELECT
  COUNT(*) AS total_trips
FROM
  trips
WHERE
  DATE(pickup_datetime) = '2025-03-01';
"""
pd.read_sql_query(query_1, db)


### 2.2 Top 5 Most Profitable Trips
The Story: The finance team wants to identify the top 5 trips that generated the most revenue (fare + tips).

In [ ]:
# --- Q2: What were our top 5 most profitable trips on March 1st, 2025? ---
query_2 = """
SELECT
  pickup_datetime,
  PULocationID,
  DOLocationID,
  trip_miles,
  base_passenger_fare + tips AS total_revenue
FROM
  trips
WHERE
  DATE(pickup_datetime) = '2025-03-01'
ORDER BY
  total_revenue DESC
LIMIT 5;
"""
pd.read_sql_query(query_2, db)

### 2.3 Pickup Location Ranking by Volume
The Story: The operations team wants to compare the key locations to see which is the busiest.

In [ ]:
# --- Q3: How do our key pickup locations rank in terms of trip volume? ---
query_3 = """
SELECT
  z.Borough,
  z.Zone,
  COUNT(t.PULocationID) AS number_of_pickups
FROM
  trips AS t
JOIN
  zone AS z
  ON t.PULocationID = z.LocationID
GROUP BY
  z.Borough,
  z.Zone
ORDER BY
  number_of_pickups DESC;
"""
pd.read_sql_query(query_3, db)

### 2.4 Top 5 Destination Zones
The Story: A marketing analyst wants to know the most popular destinations for targeted campaigns.

In [ ]:
# --- Q4: What are the top 5 destination zones? ---
query_4 = """
SELECT
  destination_zone.Borough AS destination_borough,
  destination_zone.Zone AS destination_zone,
  COUNT(*) AS trip_count
FROM
  trips AS t
JOIN
  zone AS destination_zone
  ON t.DOLocationID = destination_zone.LocationID
GROUP BY
  destination_borough,
  destination_zone
ORDER BY
  trip_count DESC
LIMIT 5;
"""
pd.read_sql_query(query_4, db)

### 2.5 Average Wait Time by Pickup Location
The Story: The team wants to compare wait times across key zones to find performance issues.

In [ ]:
# --- Q5: What is the average wait time (request to on-scene) for each location? ---
query_5 = """
SELECT
  z.Zone,
  AVG(
    strftime('%s', t.on_scene_datetime) - strftime('%s', t.request_datetime)
  ) / 60.0 AS avg_wait_time_minutes
FROM
  trips AS t
JOIN
  zone AS z
  ON t.PULocationID = z.LocationID
WHERE
  t.on_scene_datetime IS NOT NULL
GROUP BY
  z.Zone
ORDER BY
  avg_wait_time_minutes DESC;
"""
pd.read_sql_query(query_5, db)


### 2.6 Peak Request Hours
The Story: The driver supply manager needs to identify the busiest hours of the day.

In [ ]:
# --- Q6: What are the peak request hours? ---
query_6 = """
WITH
  HourlyTrips AS (
    SELECT
      CAST(strftime('%H', request_datetime) AS INTEGER) AS hour_of_day
    FROM
      trips
  )
SELECT
  hour_of_day,
  COUNT(*) AS number_of_trips
FROM
  HourlyTrips
GROUP BY
  hour_of_day
ORDER BY
  number_of_trips DESC;
"""
pd.read_sql_query(query_6, db)

### 2.7 Busiest Day of the Week
The Story: Operations needs to know which day has the highest demand for scheduling.

In [ ]:
# --- Q7: What is the busiest day of the week? ---
query_7 = """
SELECT
  CASE CAST (strftime('%w', pickup_datetime) AS INTEGER)
    WHEN 0 THEN 'Sunday'
    WHEN 1 THEN 'Monday'
    WHEN 2 THEN 'Tuesday'
    WHEN 3 THEN 'Wednesday'
    WHEN 4 THEN 'Thursday'
    WHEN 5 THEN 'Friday'
    ELSE 'Saturday'
  END AS day_of_week,
  COUNT(*) AS number_of_trips
FROM
  trips
GROUP BY
  day_of_week
ORDER BY
  number_of_trips DESC;
"""
pd.read_sql_query(query_7, db)

### 2.8 Morning vs. Evening Rush Hour Comparison
The Story: The pricing team wants to compare demand and revenue between commute times.

In [ ]:
# --- Q8: How do morning rush vs. evening rush hours compare? ---
query_8 = """
WITH
  TripTimes AS (
    SELECT
      base_passenger_fare,
      CASE
        WHEN CAST(strftime('%H', pickup_datetime) AS INTEGER) BETWEEN 7
        AND 9 THEN 'Morning Rush'
        WHEN CAST(strftime('%H', pickup_datetime) AS INTEGER) BETWEEN 16
        AND 18 THEN 'Evening Rush'
        ELSE 'Off-Peak'
      END AS time_period
    FROM
      trips
  )
SELECT
  time_period,
  COUNT(*) AS number_of_trips,
  AVG(base_passenger_fare) AS average_fare
FROM
  TripTimes
WHERE
  time_period != 'Off-Peak'
GROUP BY
  time_period;
"""
pd.read_sql_query(query_8, db)

### 2.9 Top 3 Drivers by Pay per App
The Story: We need to identify top-earning drivers for a bonus program.

In [ ]:
# --- Q9: Who are the top 3 performing drivers in terms of total pay per app? ---
query_9 = """
WITH
  DriverPayRank AS (
    SELECT
      t.dispatching_base_num,
      c."App Company Affiliation" AS app_company,
      SUM(t.driver_pay) AS total_pay,
      DENSE_RANK() OVER (
        PARTITION BY
          c."App Company Affiliation"
        ORDER BY
          SUM(t.driver_pay) DESC
      ) AS pay_rank
    FROM
      trips AS t
    JOIN
      company AS c
      ON t.hvfhs_license_num = c."High Volume License Number"
    GROUP BY
      t.dispatching_base_num,
      app_company
  )
SELECT
  app_company,
  dispatching_base_num,
  total_pay
FROM
  DriverPayRank
WHERE
  pay_rank <= 3;
"""
pd.read_sql_query(query_9, db)

### 2.10 Fare Anomaly Detection
The Story: The pricing team wants to find fare anomalies for specific routes.

In [ ]:
# --- Q10: How does each trip's fare compare to the average for its route? ---
query_10 = """
WITH
  TripFaresWithBoroughs AS (
    SELECT
      t.PULocationID,
      pz.Borough AS pickup_borough,
      dz.Borough AS dropoff_borough,
      t.base_passenger_fare
    FROM
      trips AS t
    JOIN
      zone AS pz
      ON t.PULocationID = pz.LocationID
    JOIN
      zone AS dz
      ON t.DOLocationID = dz.LocationID
  )
SELECT
  tf.pickup_borough,
  tf.dropoff_borough,
  tf.base_passenger_fare,
  AVG(tf.base_passenger_fare) OVER (
    PARTITION BY
      tf.pickup_borough,
      tf.dropoff_borough
  ) AS avg_fare_for_route,
  tf.base_passenger_fare - AVG(tf.base_passenger_fare) OVER (
    PARTITION BY
      tf.pickup_borough,
      tf.dropoff_borough
  ) AS fare_difference
FROM
  TripFaresWithBoroughs AS tf
ORDER BY
  ABS(fare_difference) DESC
LIMIT 10;
"""
pd.read_sql_query(query_10, db)